In [62]:
import math
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.python.framework import ops
import pandas as pd
import time
#import matplotlib.pyplot as plt
#%matplotlib inline

np.random.seed(1)

In [2]:
# LOAD FILES

# prefix string with r to avoid unicode escape on '\U'
path = r'C:\Users\Nick Bashour\Documents\Personal\14. Stanford\2. Academics\3. 2021 Spring\1. CS 230\2. Project\2. Data\MachineLearningCVE\\'
files = [
    'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',
    'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
    'Friday-WorkingHours-Morning.pcap_ISCX.csv',
    'Monday-WorkingHours.pcap_ISCX.csv',
    'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
    'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
    'Tuesday-WorkingHours.pcap_ISCX.csv',
    'Wednesday-workingHours.pcap_ISCX.csv'
]

for i in range(len(files)):    
    files[i] = path + files[i]

pd_files = [pd.read_csv(f) for f in files]
df = pd.concat(pd_files)
df.columns = df.columns.str.strip()

In [3]:
labels = df['Label'].unique()
labels

array(['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration',
       'Web Attack - Brute Force', 'Web Attack - XSS',
       'Web Attack - Sql Injection', 'FTP-Patator', 'SSH-Patator',
       'DoS slowloris', 'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye',
       'Heartbleed'], dtype=object)

In [41]:
# RANDOMLY SORT & VECTORIZE DATA. SPLIT INTO TEST / TRAIN, AND NORMALIZE ACCORDING TO MEAN / STD OF TRAIN.
# Note: due to compute constraints, for the project milestone I am only processing data for the 
#       first of 8 files within the data set.

tic = time.time()

# Random sort
data = pd_files[0].sample(frac=1)

# dimensions
m = len(data)
nx = len(data.columns)-1
ny = len(labels)
m_test = 10000
m_train = m - m_test

# blank arrays
X_train = np.zeros((nx, m_train))
Y_train = np.zeros((ny, m_train))
X_test = np.zeros((nx, m_test))
Y_test = np.zeros((ny, m_test))

# Convert labels into a one hot vector
def one_hot(label):    
    return np.where(labels == label, 1, 0)

# Vectorize training set
for j in range(m_train):
    for i in range(nx):
        X_train[i,j] = data.iloc[j,i]
    Y_train[:,j] = one_hot(str(data.iloc[j,len(data.columns)-1]))
    
# Vectorize test set
for j in range(m_test):
    for i in range(nx):
        X_test[i,j] = data.iloc[j+m_train,i]
    Y_test[:,j] = one_hot(str(data.iloc[j+m_train,len(data.columns)-1]))
    
toc = time.time()
print("time elapsed: " + str(toc-tic))

time elapsed: 395.6536331176758


In [49]:
# Check for Nan's and replace with 0's
# print(np.count_nonzero(np.isnan(X_train)))
# X_train[np.isnan(X_train)] = 0
# print(np.count_nonzero(np.isnan(X_train)))

4
0


In [50]:
# Normalize training & test sets according to training set data
mean = np.mean(X_train, axis=1)
epsilon = pow(10,-7)
std = np.std(X_train, axis=1) + epsilon
mean = mean.reshape(mean.shape[0],1)
std = std.reshape(std.shape[0],1)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

<ipython-input-50-05765efc8020>:8: RuntimeWarning: invalid value encountered in subtract
  X_train = (X_train - mean)/std
<ipython-input-50-05765efc8020>:9: RuntimeWarning: invalid value encountered in subtract
  X_test = (X_test - mean)/std


In [60]:
# Check for Nan's and replace with 0's
# print(np.count_nonzero(np.isnan(X_test)))
# X_test[np.isnan(X_test)] = 0
# print(np.count_nonzero(np.isnan(X_test)))

20000
0


In [61]:
# Write files to CSV

out_X_train = open("out_X_train.csv", "w")
np.savetxt('out_X_train.csv', X_train, delimiter=',')
out_Y_train = open("out_Y_train.csv", "w")
np.savetxt('out_Y_train.csv', Y_train, delimiter=',')

out_X_test = open("out_X_test.csv", "w")
np.savetxt('out_X_test.csv', X_test, delimiter=',')
out_Y_test = open("out_Y_test.csv", "w")
np.savetxt('out_Y_test.csv', Y_test, delimiter=',')